In [1]:
import os
os.chdir("/workspaces/dev/modules")

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [3]:
MODEL_ID = "openchat/openchat-3.6-8b-20240522"
CACHE_DIR = "/workspaces/dev/_Shared/Models/openChat/openchat-3.6-8b-20240522"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=CACHE_DIR)
model = AutoModelForCausalLM.from_pretrained(
  MODEL_ID, 
  cache_dir=CACHE_DIR,
  torch_dtype=torch.float16,
  device_map="auto"
)

2025-03-21 18:06:01.885404: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-21 18:06:01.895535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742580361.911055   15919 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742580361.914428   15919 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 18:06:01.924868: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [5]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
messages = [
    {"role": "user", "content": "Explain how large language models work in detail."},
]
inputs = tokenizer.apply_chat_template(
    messages, 
    add_generation_prompt=True, 
    return_dict=True,
    return_tensors="pt",
    padding = True
)

inputs['attention_mask'] = (inputs['input_ids'] != tokenizer.pad_token_id).long()
inputs = {k : v.to(model.device) for k, v in inputs.items()}

outputs = model.generate(
    **inputs,
    do_sample=True,
    temperature=0.5,
    max_new_tokens=1024
)
response = outputs[0][inputs.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 